# 特征选择
数据和特征决定了机器学习的上限，而模型和算法只是逼近这个上限而已。我们聚焦sklearn中的feature_selection库来进行特征选择。通常来说从两方面考虑来选择特征：<br>
特征是否发散：如果不发散，方差接近0,也就是说样本在这个特征上没有差异，即这个特征对样本的区分并没有什么用。<br>
特征与目标的相关性：与目标相关性高的特征应该优先选择。

## 一、过滤法
按照发散性或者相关性对各个特征进行评分，设定阈值或者待选择阈值的个数，选择特征。


In [49]:
from sklearn import datasets
iris=datasets.load_iris()

### 方差选择法
使用方差选择法，计算各个特征的方差，然后根据阈值，选择方差大于阈值的特征。

In [50]:
from sklearn.feature_selection import VarianceThreshold

In [51]:
var=VarianceThreshold(threshold=0)#调用选择器
variance=var.fit_transform(iris.data)
print(variance[:10])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]]


### 相关系数法、卡方检验
计算各个特征值对于目标值的相关系数。使用feature_selction库的SelectionKBest类。相关系数只适合连续特征的选择。

In [52]:
import numpy as np
from sklearn.feature_selection import SelectKBest
from scipy.stats import pearsonr
from sklearn.feature_selection import chi2   #引入卡方检验的函数

#定义评估函数,得到每个输入特征与最后结果的相关性。函数输出值为一个列表。map函数相当于一个for循环
score_func=lambda X,Y:np.array(list(map(lambda x:pearsonr(x,Y)[0],X.T))).T  
SelectKBest(score_func,k=2).fit_transform(iris.data,iris.target)[:10]#k为选择的特征个数

array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.7, 0.4],
       [1.4, 0.3],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.5, 0.1]])

In [53]:
SelectKBest(chi2,k=2).fit_transform(iris.data,iris.target)[:10]#k为选择的特征个数

array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.7, 0.4],
       [1.4, 0.3],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.5, 0.1]])

### 互信息法


In [55]:
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn import metrics
mic=metrics.mutual_info_score
score_func=lambda X,Y:np.array(list(map(lambda x:mic(x,Y),X.T))).T
SelectKBest(score_func,k=2).fit_transform(iris.data,iris.target)[:10]    

array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.7, 0.4],
       [1.4, 0.3],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.5, 0.1]])

## 二、包装法
递归特征消除法（RFE）:<br>
使用一个基模型来进行多轮训练，每轮训练后，消除若干权重系数的特征，在基于新的特征进行下一轮训练。

In [57]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [58]:
RFE(estimator=LogisticRegression(),n_features_to_select=2).fit_transform(iris.data,iris.target)[:10]

d:\py\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\py\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\py\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\py\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\py\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarn

array([[3.5, 0.2],
       [3. , 0.2],
       [3.2, 0.2],
       [3.1, 0.2],
       [3.6, 0.2],
       [3.9, 0.4],
       [3.4, 0.3],
       [3.4, 0.2],
       [2.9, 0.2],
       [3.1, 0.1]])